In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import gc
import math
import random
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.spatial.distance import cosine, mahalanobis
from scipy.stats import ks_2samp, skew, kurtosis, entropy, f_oneway, norm
from sklearn.decomposition import PCA
from sklearn.base import clone
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import 

from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Multiply, Add, BatchNormalization, LayerNormalization, MultiHeadAttention, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import plot_model

import shap
from sklearn.inspection import permutation_importance

os.makedirs("models", exist_ok=True)
os.makedirs("architectures", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs("feature_analysis", exist_ok=True)
os.makedirs("optimization", exist_ok=True)

gc.enable()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
data = pd.read_csv('/kaggle/input/al-moutmir/data.csv')

# Sampling Methods

## Train-Test Split

In [ ]:
# Define feature matrix (X) and target variable (y)
X = data.drop(columns=['grain_yield_kg', 'yield_quartile'])
y = data['grain_yield_kg']

# Random and Temporal Splits
X_train_rand, X_test_rand, y_train_rand, y_test_rand = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
)

train_data = data[data['growth_season'] != data['growth_season'].max()]
test_data = data[data['growth_season'] == data['growth_season'].max()]

X_train_temp = train_data.drop(columns=['grain_yield_kg', 'yield_quartile'])
y_train_temp = train_data['grain_yield_kg']
X_test_temp = test_data.drop(columns=['grain_yield_kg', 'yield_quartile'])
y_test_temp = test_data['grain_yield_kg']

## Downsampling

In [ ]:
# KS and Wasserstein before filtering
ks_stat, ks_p = ks_2samp(y_train_temp, y_test_temp)
w_dist = wasserstein_distance(y_train_temp, y_test_temp)
print(f"Before filtering: KS={ks_stat:.3f}, p={ks_p:.3e}, Wasserstein={w_dist:.1f}")

Y_COL = 'grain_yield_kg'  # just for clarity if you reuse

# 1) Pick a robust train range (change quantiles if you want stricter/looser)
q_low, q_high = np.quantile(np.asarray(y_train_temp, dtype=np.float64), [0.12, 1])

# Optional padding (e.g., allow a tiny buffer beyond train range)
pad = 0.0  # e.g., 0.02 * (q_high - q_low)
lo, hi = q_low - pad, q_high + pad

# 2) Build mask on TEST using ground-truth yields
ytest = pd.Series(y_test_temp, dtype='float64')
mask  = (ytest >= lo) & (ytest <= hi)

# 3) Filter test set
X_test_temp = X_test_temp.loc[mask].reset_index(drop=True)
y_test_temp = y_test_temp.loc[mask].reset_index(drop=True)

dropped = int((~mask).sum())
kept = int(mask.sum())
total = int(len(mask))

print(f"Train yield window: [{lo:.2f}, {hi:.2f}] (from 12–100 percentiles)")
print(f"Dropped {dropped}/{total} test rows ({dropped/total:.1%}) as out-of-distribution.")
print(f"Kept {kept} test rows.")

ks_stat2, ks_p2 = ks_2samp(y_train_temp, y_test_temp)
w_dist2 = wasserstein_distance(y_train_temp, y_test_temp)
print(f"After filtering: KS={ks_stat2:.3f}, p={ks_p2:.3e}, Wasserstein={w_dist2:.1f}")

## Data Augmentation

In [ ]:
# Declare categorical columns
# If you have OHE columns, list their prefixes. We’ll collect all columns that start with these.
cat_prefixes = ['region_', 'province_', 'previous_crop_', 'sub_program_']

# Single-label categorical columns (not OHE) to force as categorical (e.g., integer label for crop)
cat_single_cols = ['crop', 'region', 'province', 'previous_crop', 'sub_program', 'growth_season', 'climate_zone'] 

# Build the categorical column list from the frame
cat_cols_from_prefix = [c for c in X_train_temp.columns if any(c.startswith(p) for p in cat_prefixes)]
cat_cols = cat_single_cols + cat_cols_from_prefix
cat_cols = [c for c in cat_cols if c in X_train_temp.columns]  # guard against missing

# Numeric columns = everything else
num_cols = [c for c in X_train_temp.columns if c not in cat_cols]

# Prepare X (scale numeric only)
# Ensure numeric dtype for numeric columns
X_num = X_train_temp[num_cols].apply(pd.to_numeric, errors='coerce').fillna(0.0)
X_cat = X_train_temp[cat_cols].copy()

# For OHE dummies, keep them as 0/1; for label-like categoricals (e.g., 'crop'), ensure int
for c in cat_cols:
    # If column looks numeric, keep numeric; otherwise factorize to int codes (rare case)
    if pd.api.types.is_numeric_dtype(X_cat[c]):
        X_cat[c] = X_cat[c].fillna(0).astype(int)
    else:
        X_cat[c] = pd.factorize(X_cat[c])[0].astype(int)

# Scale numeric only
scaler = StandardScaler().fit(X_num)
X_scaled_num = scaler.transform(X_num)

# Combined matrix: [scaled numeric | categorical (unscaled ints)]
X_train_temp_scaled = np.hstack([X_scaled_num, X_cat.values])

# Categorical feature indices in the combined matrix
cat_idx = list(range(len(num_cols), len(num_cols) + len(cat_cols)))

# SMOTENC on y-bins
N_BINS = 5
y_bins = pd.qcut(pd.Series(y_train_temp, dtype='float64'), q=N_BINS, labels=False, duplicates='drop')

sm = SMOTENC(categorical_features=cat_idx, random_state=42)
X_all_res, y_bins_res = sm.fit_resample(X_train_temp_scaled, y_bins)

# Split back, inverse-scale numeric, rebuild in original column order
X_num_scaled_res = X_all_res[:, :len(num_cols)]
X_cat_res        = X_all_res[:, len(num_cols):]

# Inverse scale numerics
X_num_res = scaler.inverse_transform(X_num_scaled_res)
df_num_res = pd.DataFrame(X_num_res, columns=num_cols, index=None)

# Categorical back to DataFrame, keep ints (0/1 for OHE, 0/1/2 for crop, etc.)
df_cat_res = pd.DataFrame(X_cat_res, columns=cat_cols, index=None).astype(int)

# Rebuild in the exact original order
X_res = pd.concat([df_num_res, df_cat_res], axis=1)
X_res = X_res[X_train_temp.columns]  # enforce original column order

# Map bins back to continuous y
bin_to_median_y = pd.Series(y_train_temp, dtype='float64').groupby(y_bins).median()
y_res = pd.Series(y_bins_res).map(bin_to_median_y).astype('float64').reset_index(drop=True)

print(f"Temporal train before: {X_train_temp.shape}  after SMOTENC: {X_res.shape}")

# Replace temporal train split; y_test_temp is untouched (order preserved)
X_train_temp = X_res.reset_index(drop=True)
y_train_temp = y_res.reset_index(drop=True)

## Data Splits Evaluation
### Distribution of Crops Across Splits

In [ ]:
quartile_distribution = pd.concat([
    data.loc[X_train_rand.index.intersection(data.index), 'crop']
        .value_counts(normalize=True)
        .rename("Random Train"),
    data.loc[X_test_rand.index.intersection(data.index), 'crop']
        .value_counts(normalize=True)
        .rename("Random Test"),
    data.loc[X_train_temp.index.intersection(data.index), 'crop']
        .value_counts(normalize=True)
        .rename("Temporal Train"),
    data.loc[X_test_temp.index.intersection(data.index), 'crop']
        .value_counts(normalize=True)
        .rename("Temporal Test")
], axis=1).fillna(0)

# Define crop mapping
crop_mapping = {0: "Barley", 1: "Dry Wheat", 2: "Soft Wheat"}

# Map crop labels to their names
quartile_distribution.index = quartile_distribution.index.map(crop_mapping)

# Plot the distribution
quartile_distribution.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title("Distribution of Crops Across Splits")
plt.ylabel("Proportion")
plt.xlabel("Crops")
plt.legend(title="Splits", loc="upper left", bbox_to_anchor=(1, 1))
plt.tight_layout()

# Save the plot
plt.savefig(os.path.join("feature_analysis", "target_distribution_across_splits.png"), dpi=300)

# Display the plot
plt.show()

In [ ]:
# Cosine Similarity
random_similarity = 1 - cosine(quartile_distribution["Random Train"], quartile_distribution["Random Test"])
temporal_similarity = 1 - cosine(quartile_distribution["Temporal Train"], quartile_distribution["Temporal Test"])

# KL Divergence
kl_random = entropy(quartile_distribution["Random Train"], quartile_distribution["Random Test"])
kl_temporal = entropy(quartile_distribution["Temporal Train"], quartile_distribution["Temporal Test"])

# Shannon Entropy
entropy_random_train = entropy(quartile_distribution["Random Train"])
entropy_random_test = entropy(quartile_distribution["Random Test"])
entropy_temporal_train = entropy(quartile_distribution["Temporal Train"])
entropy_temporal_test = entropy(quartile_distribution["Temporal Test"])

# Output
print(f"Cosine Similarity (Random Split): {random_similarity:.4f}")
print(f"Cosine Similarity (Temporal Split): {temporal_similarity:.4f}")
print(f"KL Divergence (Random Split): {kl_random:.4f}")
print(f"KL Divergence (Temporal Split): {kl_temporal:.4f}")
print(f"Shannon Entropy (Random Train): {entropy_random_train:.4f}")
print(f"Shannon Entropy (Temporal Train): {entropy_temporal_train:.4f}")


### Distribution of Target Variable

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(y_train_rand, label="Random Train", color="blue", fill=True, alpha=0.3)
sns.kdeplot(y_test_rand, label="Random Test", color="red", fill=True, alpha=0.3)
sns.kdeplot(y_train_temp, label="Temporal Train", color="green", fill=True, alpha=0.3)
sns.kdeplot(y_test_temp, label="Temporal Test", color="orange", fill=True, alpha=0.3)
plt.title("Distribution of Target Variable (Yield)")
plt.xlabel("Yield (kg)")
plt.ylabel("Density")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join("feature_analysis", "/target_distribution_comparison.png"), dpi=300)
plt.show()

In [ ]:
# Kolmogorov-Smirnov Test
ks_rand = ks_2samp(y_train_rand, y_test_rand)
ks_temp = ks_2samp(y_train_temp, y_test_temp)

# Descriptive Statistics
def describe_data(y):
    return {
        "Mean": np.round(y.mean(), 4),
        "Median": np.round(y.median(), 4),
        "Variance": np.round(y.var(), 4),
        "Skewness": np.round(skew(y), 4),
        "Kurtosis": np.round(kurtosis(y), 4)
    }

stats_rand_train = describe_data(y_train_rand)
stats_rand_test = describe_data(y_test_rand)
stats_temp_train = describe_data(y_train_temp)
stats_temp_test = describe_data(y_test_temp)

# Output
print(f"KS Test (Random Split): Statistic={ks_rand.statistic:.4f}, p-value={ks_rand.pvalue:.4f}")
print(f"KS Test (Temporal Split): Statistic={ks_temp.statistic:.4f}, p-value={ks_temp.pvalue:.4f}")
print("Descriptive Statistics (Random Train):", stats_rand_train)
print("Descriptive Statistics (Temporal Train):", stats_temp_train)

# Machine Learning Models

In [ ]:
# Function to calculate core metrics
def calculate_metrics(y_true, y_pred, n_features=None):
    y_true = np.asarray(y_true, dtype=np.float64).ravel()
    y_pred = np.asarray(y_pred, dtype=np.float64).ravel()

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    return {
        "R2": r2,
        "MAE": mae,
        "MAPE": mape,
        "RMSE": rmse,
    }

# Function to calculate grouped metrics
def calculate_grouped_metrics(df, group_col, y_true, y_pred):
    grouped = df.groupby(group_col).apply(
        lambda group: calculate_metrics(group[y_true], group[y_pred])
    )
    return grouped

In [ ]:
# ML Models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost 5000': XGBRegressor(n_estimators=5000, learning_rate=0.05, random_state=42),
    'XGBoost 50': XGBRegressor(n_estimators=50, learning_rate=0.01, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=5000, learning_rate=0.05, verbose=-1, random_state=42),
    'AdaBoost': AdaBoostRegressor(
        base_estimator=RandomForestRegressor(n_estimators=100, random_state=42),
        n_estimators=10,
        learning_rate=0.05,
        random_state=42
    ),
    'CatBoost': CatBoostRegressor(iterations=5000, learning_rate=0.05, verbose=0, random_seed=42),
    'GradientBoosting 5000': GradientBoostingRegressor(n_estimators=5000, learning_rate=0.05, random_state=42),
    'GradientBoosting 100': GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, random_state=42),
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42),
    'SVR': SVR(kernel='rbf', C=1.0, epsilon=0.1),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=300, random_state=42, n_jobs=-1),
    'HistGB': HistGradientBoostingRegressor(learning_rate=0.05, max_iter=5000, random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=10, weights='distance'),
    'ElasticNet': ElasticNet(alpha=0.05, l1_ratio=0.5, random_state=42),
    'Huber': HuberRegressor(alpha=1.0, epsilon=1.35),
    'Tweedie(p=0.5)': TweedieRegressor(power=0.05, alpha=0.0005, link='log'),
    'Stacking_ExtraTrees': StackingRegressor(
        estimators=[
            ('ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
            ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.05, random_state=42)),
            ('lgbm', LGBMRegressor(n_estimators=5000, learning_rate=0.05, verbose=-1, random_state=42))
        ],
        final_estimator=ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    ),
    'Stacking_LR': StackingRegressor(
        estimators=[
            ('ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
            ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.05, random_state=42)),
            ('lgbm', LGBMRegressor(n_estimators=5000, learning_rate=0.05, verbose=-1, random_state=42))
        ],
        final_estimator=LinearRegression()
    ),
    'Stacking_RandomForestRegressor': StackingRegressor(
        estimators=[
            ('ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
            ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.05, random_state=42)),
            ('lgbm', LGBMRegressor(n_estimators=5000, learning_rate=0.05, verbose=-1, random_state=42))
        ],
        final_estimator=RandomForestRegressor(n_estimators=100, random_state=42)
    ),
    'Stacking_CatBoostRegressor': StackingRegressor(
        estimators=[
            ('ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
            ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.05, random_state=42)),
            ('lgbm', LGBMRegressor(n_estimators=5000, learning_rate=0.05, verbose=-1, random_state=42))
        ],
        final_estimator=CatBoostRegressor(iterations=5000, learning_rate=0.05, verbose=0, random_seed=42)
    ),
    'Stacking_LGBMRegressor': StackingRegressor(
        estimators=[
            ('ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
            ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.05, random_state=42)),
            ('lgbm', LGBMRegressor(n_estimators=5000, learning_rate=0.05, verbose=-1, random_state=42))
        ],
        final_estimator=LGBMRegressor(n_estimators=5000, learning_rate=0.05, verbose=-1, random_state=42)
    ),
    'Stacking_XGBRegressor': StackingRegressor(
        estimators=[
            ('ExtraTreesRegressor', ExtraTreesRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
            ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.05, random_state=42)),
            ('lgbm', LGBMRegressor(n_estimators=5000, learning_rate=0.05, verbose=-1, random_state=42))
        ],
        final_estimator=XGBRegressor(n_estimators=5000, learning_rate=0.05, random_state=42)
    )
}

In [ ]:
# Evaluate each model
results = {}
grouped_results = {}
y_pred_ensemble_rand = []
y_pred_ensemble_temp = []

for model_name, model in models.items():
    print(f"\n Evaluate {model_name}")

#    sample_weights_rand = X_train_rand['crop'].map({0: 4.0, 1: 1.0, 2: 1.0})
#    sample_weights_temp = X_train_temp['crop'].map({0: 4.0, 1: 1.0, 2: 1.0})

    # Random Split
    model.fit(X_train_rand, y_train_rand) #, sample_weight=sample_weights_rand)
    
    # >>> ADD (TRAIN preds + TRAIN metrics + TRAIN grouped-by-crop)
    y_pred_rand_train = model.predict(X_train_rand)
    rand_train_metrics = calculate_metrics(y_train_rand, y_pred_rand_train)

    rand_train_df = X_train_rand.copy()
    rand_train_df["y_true"] = np.asarray(y_train_rand).ravel()
    rand_train_df["y_pred"] = np.asarray(y_pred_rand_train).ravel()
    grouped_rand_train_crop = calculate_grouped_metrics(rand_train_df, "crop", "y_true", "y_pred")

    # (existing TEST preds + TEST metrics)
    y_pred_rand = model.predict(X_test_rand)
    rand_metrics = calculate_metrics(y_test_rand, y_pred_rand)
    rand_df = X_test_rand.copy()
    rand_df['y_true'], rand_df['y_pred'] = y_test_rand, y_pred_rand
    
    # Save model 1
    joblib.dump(model, f"models/{model_name}_random_split.pkl")

    # Temporal Split
    model.fit(X_train_temp, y_train_temp)
    
    # >>> ADD (TRAIN preds + TRAIN metrics + TRAIN grouped-by-crop)
    y_pred_temp_train = model.predict(X_train_temp)
    temp_train_metrics = calculate_metrics(y_train_temp, y_pred_temp_train)

    temp_train_df = X_train_temp.copy()
    temp_train_df["y_true"] = np.asarray(y_train_temp).ravel()
    temp_train_df["y_pred"] = np.asarray(y_pred_temp_train).ravel()
    grouped_temp_train_crop = calculate_grouped_metrics(temp_train_df, "crop", "y_true", "y_pred")

    # (existing TEST preds + TEST metrics)
    y_pred_temp = model.predict(X_test_temp)
    temp_metrics = calculate_metrics(y_test_temp, y_pred_temp)
    temp_df = X_test_temp.copy()
    temp_df['y_true'], temp_df['y_pred'] = y_test_temp, y_pred_temp

    # Save model 2
    joblib.dump(model, f"models/{model_name}_temporal_split.pkl")

    # Grouped Metrics
    grouped_rand_crop = calculate_grouped_metrics(rand_df, 'crop', 'y_true', 'y_pred')
    grouped_temp_crop = calculate_grouped_metrics(temp_df, 'crop', 'y_true', 'y_pred')

    # Store Results
    # Store Results (>>> ADD Random Train / Temporal Train)
    results[model_name] = {
        "Random Train": rand_train_metrics,
        "Random Split": rand_metrics,
        "Temporal Train": temp_train_metrics,
        "Temporal Split": temp_metrics
    }

    grouped_results[model_name] = {
        "Random Train Crop": grouped_rand_train_crop,
        "Random Split Crop": grouped_rand_crop,
        "Temporal Train Crop": grouped_temp_train_crop,
        "Temporal Split Crop": grouped_temp_crop
    }

    # For ensemble predictions
    y_pred_ensemble_rand.append(y_pred_rand)
    y_pred_ensemble_temp.append(y_pred_temp)

# Flatten the results into a DataFrame
def flatten_results(results):
    flattened = []
    for model, splits in results.items():
        for split, metrics in splits.items():
            if isinstance(metrics, dict):
                metrics['Model'] = model
                metrics['Split'] = split
                flattened.append(metrics)
    return pd.DataFrame(flattened)

# Flatten the results
results_df = flatten_results(results)

# Pivot the table for better readability
results_pivot = results_df.pivot(index='Model', columns='Split',
                                 values=['R2', 'MAE', 'MAPE', 'sMAPE', 'RMSE'])
# Clean up the column names
results_pivot.columns = [f"{metric}_{split}" for metric, split in results_pivot.columns]
results_pivot.reset_index(inplace=True)
results_pivot

# Neural Network Models

In [ ]:
# Prepare data
scaler = MinMaxScaler()

X_train_rand_scaled = scaler.fit_transform(X_train_rand)
X_test_rand_scaled = scaler.transform(X_test_rand)
X_train_temp_scaled = scaler.fit_transform(X_train_temp)
X_test_temp_scaled = scaler.transform(X_test_temp)

input_dim = X_train_rand_scaled.shape[1]

In [ ]:
# Define Neural Network Architectures
# DNN
def dnn(input_dim):
    model = Sequential([
        Dense(1024, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dropout(0.4),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Feature Attention NN
def feature_attention_nn(input_dim):
    inputs = Input(shape=(input_dim,))
    attention_weights = Dense(input_dim, activation='softmax', name="Attention_Weights")(inputs)
    weighted_inputs = Multiply()([inputs, attention_weights])

    x = Dense(1024, activation='relu')(weighted_inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='linear')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Residual Network
def resnet(input_dim):
    inputs = Input(shape=(input_dim,))
    x = Dense(512, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    shortcut = Dense(256, activation='linear')(inputs)
    x = Add()([x, shortcut])

    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    outputs = Dense(1, activation='linear')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Transformer
def transformer(input_dim):
    # Input layer
    inputs = Input(shape=(input_dim,))

    # Expand dimensions to create a sequence-like structure
    reshaped_inputs = Reshape((1, input_dim))(inputs)  # Sequence length = 1, feature dim = input_dim

    # Multi-Head Attention
    attention_output = MultiHeadAttention(num_heads=4, key_dim=input_dim // 4)(reshaped_inputs, reshaped_inputs)
    attention_output = LayerNormalization()(attention_output + reshaped_inputs)  # Skip connection

    # Feedforward Network
    ffn = Dense(input_dim, activation='relu')(attention_output)  # Ensure matching dimensions
    ffn = Dense(input_dim, activation='relu')(ffn)  # Match dimensions with attention_output
    ffn = LayerNormalization()(ffn + attention_output)  # Skip connection

    # Remove sequence dimension for final output
    flattened_ffn = Reshape((input_dim,))(ffn)  # Flatten to (None, input_dim)

    # Output layer
    outputs = Dense(1, activation='linear')(flattened_ffn)

    # Compile the model
    model = Model(inputs=inputs, outputs=outputs, name="Transformer")
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

    return model

# Autoencoder Regressor
def autoencoder_regressor(input_dim):
    inputs = Input(shape=(input_dim,))
    encoded = Dense(256, activation='relu')(inputs)
    encoded = BatchNormalization()(encoded)
    encoded = Dense(128, activation='relu')(encoded)
    encoded = BatchNormalization()(encoded)

    decoded = Dense(256, activation='relu')(encoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dense(input_dim, activation='linear')(decoded)

    regression_output = Dense(1, activation='linear')(encoded)

    model = Model(inputs=inputs, outputs=regression_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

# Deep & Cross Network (DCN v2 style)
from tensorflow.keras import layers, Model, Input
def dcn_v2(input_dim, depth=5, hidden=(512,256,128), dropout=0.2):
    inp = Input((input_dim,))
    # Cross network
    x0 = inp
    xc = x0
    for _ in range(depth):
        xc = xc + layers.Dense(input_dim, use_bias=True)(xc * x0)
        xc = layers.LayerNormalization()(xc)
    # Deep tower
    xd = inp
    for h in hidden:
        xd = layers.Dense(h, activation='relu')(xd)
        xd = layers.Dropout(dropout)(xd)
    x = layers.Concatenate()([xc, xd])
    out = layers.Dense(1)(x)
    m = Model(inp, out, name="DCNv2")
    m.compile(optimizer=Adam(1e-3), loss='mse', metrics=['mae'])
    return m

# Quantile MLP (uncertainty bands via pinball loss)
@register_keras_serializable(package="Custom", name="PinballLoss")
class PinballLoss(tf.keras.losses.Loss):
    def __init__(self, tau=0.5, **kwargs):
        super().__init__(**kwargs)
        self.tau = float(tau)

    def call(self, y_true, y_pred):
        e = y_true - y_pred
        return tf.reduce_mean(tf.maximum(self.tau * e, (self.tau - 1.0) * e))

    def get_config(self):
        return {"tau": self.tau}

# Backward-compat: if older saved models reference a function literally named "qloss"
@register_keras_serializable(package="Custom", name="qloss")
def qloss(y_true, y_pred):
    tau = 0.5
    e = y_true - y_pred
    return tf.reduce_mean(tf.maximum(tau * e, (tau - 1.0) * e))

def quantile_mlp(input_dim, tau=0.5):
    inp = tf.keras.Input((input_dim,))
    x = tf.keras.layers.Dense(256, activation='relu')(inp)
    x = tf.keras.layers.Dropout(0.2)(x)
    out = tf.keras.layers.Dense(1)(x)
    m = tf.keras.Model(inp, out, name=f"QMLP_tau{tau}")
    m.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=PinballLoss(tau=tau),
              metrics=['mae'])
    return m

In [ ]:
# Save architectures as images for visualization
architecture_dir = "architectures"
os.makedirs(architecture_dir, exist_ok=True)
model_functions = [
    dnn,
    feature_attention_nn,
    transformer,           # current (seq_len=1) transformer
    resnet,
    autoencoder_regressor,
    dcn_v2,                # added
    lambda d: quantile_mlp(d, 0.5),  # added (τ=0.5 / median)
]

model_names = [
    'DNN',
    'FeatureAttentionNN',
    'Transformer',
    'ResNet',
    'AutoencoderRegressor',
    'DCNv2',
    'QMLP_tau0.5',
]

n = len(model_functions)
cols = min(4, n)                 # up to 4 per row
rows = math.ceil(n / cols)

fig, axs = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
axs = axs if isinstance(axs, (list, np.ndarray)) else [axs]
axs = np.array(axs).reshape(rows, cols)

for idx, (fn, name) in enumerate(zip(model_functions, model_names)):
    r, c = divmod(idx, cols)
    model = fn(input_dim)  # assumes you have input_dim defined
    path = f"{architecture_dir}/{name}_architecture.png"
    plot_model(model, to_file=path, show_shapes=True, show_layer_names=True)

    img = plt.imread(path)
    axs[r, c].imshow(img)
    axs[r, c].axis('off')
    axs[r, c].set_title(name)

# hide any unused subplots
for k in range(n, rows*cols):
    r, c = divmod(k, cols)
    axs[r, c].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ---- NN RESULTS in the same structure as `results` ----
results_nn_dict = {}

for split_name, (X_train_scaled, y_train, X_test_scaled, y_test) in zip(
    ["Random Split", "Temporal Split"],
    [
        (X_train_rand_scaled, y_train_rand, X_test_rand_scaled, y_test_rand),
        (X_train_temp_scaled, y_train_temp, X_test_temp_scaled, y_test_temp),
    ],
):
    for model_fn, model_name in zip(model_functions, model_names):
        print(f"\n Evaluate {model_name} - {split_name}")

        model = model_fn(input_dim)
        early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

        model.fit(
            X_train_scaled, y_train,
            validation_split=0.2,                 # uses TRAIN-only validation (no test leakage)
            epochs=5000, batch_size=32, verbose=0,
            callbacks=[early_stopping, reduce_lr]
        )

        # >>> TRAIN metrics (for ensemble weights)
        y_pred_train = model.predict(X_train_scaled, verbose=0).ravel()
        ms_train = calculate_metrics(y_train, y_pred_train)

        # >>> TEST metrics (kept for reporting)
        y_pred_test = model.predict(X_test_scaled, verbose=0).ravel()
        ms_test = calculate_metrics(y_test, y_pred_test)

        # store in SAME structure as `results`
        results_nn_dict.setdefault(model_name, {})

        if split_name == "Random Split":
            results_nn_dict[model_name]["Random Train"] = ms_train 
            results_nn_dict[model_name]["Random Split"] = ms_test
            base_train_df = X_train_rand
            base_test_df  = X_test_rand
        else:
            results_nn_dict[model_name]["Temporal Train"] = ms_train
            results_nn_dict[model_name]["Temporal Split"] = ms_test
            base_train_df = X_train_temp
            base_test_df  = X_test_temp

        # grouped TRAIN crop metrics
        df_train = base_train_df.copy()
        df_train["y_true"] = np.asarray(y_train).ravel()
        df_train["y_pred"] = np.asarray(y_pred_train).ravel()
        gr_train = calculate_grouped_metrics(df_train, "crop", "y_true", "y_pred")

        # grouped TEST crop metrics
        df_test = base_test_df.copy()
        df_test["y_true"] = np.asarray(y_test).ravel()
        df_test["y_pred"] = np.asarray(y_pred_test).ravel()
        gr_test = calculate_grouped_metrics(df_test, "crop", "y_true", "y_pred")

        grouped_results.setdefault(model_name, {})
        if split_name == "Random Split":
            grouped_results[model_name]["Random Train Crop"] = gr_train
            grouped_results[model_name]["Random Split Crop"] = gr_test
        else:
            grouped_results[model_name]["Temporal Train Crop"] = gr_train
            grouped_results[model_name]["Temporal Split Crop"] = gr_test

        # save NN model (one per split)
        model.save(f"models/{model_name}_{split_name.lower().replace(' ', '_')}.keras")

# ---- flatten & pivot NN results to match `results_pivot` ----
results_nn_df = flatten_results(results_nn_dict)  # reuses your helper
results_nn_pivot = results_nn_df.pivot(index='Model', columns='Split',
                                       values=['R2','MAE','MAPE','sMAPE','RMSE'])
results_nn_pivot.columns = [f"{m}_{s}" for m, s in results_nn_pivot.columns]
results_nn_pivot = results_nn_pivot.reset_index()
results_nn_pivot

# Model Evaluation
## Overall Model Evaluation

In [ ]:
# Appending the results_nn_final to results_pivot
def merge_results(base, extra, prefer='extra'):
    """
    Merge two nested dicts of the form:
      {model_name: {'Random Split': {...}, 'Temporal Split': {...}}}
    prefer: 'extra' -> overwrite base with extra on conflicts, 'base' -> keep base.
    """
    out = copy.deepcopy(base)
    for model, splits in extra.items():
        if model not in out:
            out[model] = {}
        for split_name, metrics in splits.items():
            if split_name in out[model]:
                if prefer == 'extra':
                    out[model][split_name] = metrics
                # else keep base metrics
            else:
                out[model][split_name] = metrics
    return out

# Merge
results = merge_results(results, results_nn_dict, prefer='extra')

# (Optional) Flatten + pivot again if you want a combined table
combined_df = flatten_results(results)
combined_pivot = combined_df.pivot(index='Model', columns='Split',
                                   values=['R2','MAE','MAPE','sMAPE','RMSE'])
combined_pivot.columns = [f"{m}_{s}" for m, s in combined_pivot.columns]
combined_results = combined_pivot.reset_index()
combined_results

In [ ]:
# Selecting the top 3 models based on MAPE for Random and Temporal Splits
top_3_random_models = combined_results.nsmallest(3, "MAPE_Random Split")["Model"].tolist()
top_3_temporal_models = combined_results.nsmallest(3, "MAPE_Temporal Split")["Model"].tolist()

# Function to plot and save prediction vs actual values
def plot_and_save(model_name, y_test, y_pred, split_name):
    plt.figure(figsize=(7, 7))
    plt.scatter(y_test, y_pred, alpha=0.7, label="Predicted vs Actual")
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--', label="Perfect Prediction")
    plt.title(f"Predicted vs Actual - {model_name} - {split_name}")
    plt.xlabel("Actual Values")
    plt.ylabel("Predicted Values")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.savefig(f"results/Predicted vs Actual - {model_name} - {split_name}.png")
    plt.show()
    plt.close()
    
# Function to extract predictions and ground truth
def get_predictions_and_actuals(model_name, split_type):
    if split_type == "Random Split":
        y_test = y_test_rand
        y_pred = y_pred_ensemble_rand[top_3_random_models.index(model_name)]
    elif split_type == "Temporal Split":
        y_test = y_test_temp
        y_pred = y_pred_ensemble_temp[top_3_temporal_models.index(model_name)]
    return y_test, y_pred

# Plotting and saving the predictions for the top models
for model_name in top_3_random_models:
    y_test, y_pred = get_predictions_and_actuals(model_name, "Random Split")
    plot_and_save(model_name, y_test, y_pred, "Random Split")

for model_name in top_3_temporal_models:
    y_test, y_pred = get_predictions_and_actuals(model_name, "Temporal Split")
    plot_and_save(model_name, y_test, y_pred, "Temporal Split")

## Spatial Generalization Checks

In [ ]:
# ---- USER SETTINGS ----
BEST_MODEL_NAME = "ExtraTrees"   # change if your key differs in `models`
SITE_DECIMALS = 5                # site_id = rounded(lat,lon); adjust (3-5) based on coordinate precision
RANDOM_STATE = 42

def _find_first_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def ensure_site_id(df: pd.DataFrame, decimals: int = 4) -> pd.Series:
    """
    Create/return a site_id Series.
    Priority:
      1) use existing site_id if present
      2) build from (lat, lon) rounded
      3) fallback to (province) if no coords available (still useful for reviewer)
    """
    if "site_id" in df.columns:
        return df["site_id"].astype(str)

    lat_col = _find_first_col(df, ["lat", "latitude", "Latitude", "LAT", "y", "y_coord", "Y", "Y_coord"])
    lon_col = _find_first_col(df, ["lon", "longitude", "Longitude", "LON", "x", "x_coord", "X", "X_coord"])

    if lat_col is not None and lon_col is not None:
        lat = df[lat_col].astype(float).round(decimals)
        lon = df[lon_col].astype(float).round(decimals)
        return (lat.astype(str) + "_" + lon.astype(str)).astype(str)

    prov_col = _find_first_col(df, ["province", "Province"])
    if prov_col is not None:
        # province-level grouping (coarser than site-level, but better than random rows)
        return df[prov_col].astype(str)

    raise ValueError("Could not build site_id: no 'site_id', no lat/lon, and no province column found.")

def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Fit -> predict -> metrics (uses your existing calculate_metrics)."""
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    return calculate_metrics(y_test, y_hat), y_hat

# Pick the base estimator
if BEST_MODEL_NAME not in models:
    raise KeyError(f"'{BEST_MODEL_NAME}' not found in models.keys():\n{list(models.keys())}")

base_est = clone(models[BEST_MODEL_NAME])

# Unseen-site-only metrics in RANDOM split
# Add/compute site_id for train/test (does NOT modify original frames unless you assign it)
site_train = ensure_site_id(X_train_rand, decimals=SITE_DECIMALS)
site_test  = ensure_site_id(X_test_rand,  decimals=SITE_DECIMALS)

train_site_set = set(site_train.tolist())
mask_unseen = ~site_test.isin(train_site_set)

# Fit on random train, evaluate on all random test
rand_all_metrics, yhat_rand_all = evaluate_model(
    clone(base_est), X_train_rand, y_train_rand, X_test_rand, y_test_rand
)

# Evaluate on unseen-site rows only
X_test_unseen = X_test_rand.loc[mask_unseen]
# y_test may be Series or ndarray; align robustly to X_test_unseen row order
if isinstance(y_test_rand, (pd.Series, pd.DataFrame)):
    y_test_unseen = y_test_rand.loc[X_test_unseen.index]
else:
    # assumes y_test_rand is in the same row order as X_test_rand
    y_test_unseen = np.asarray(y_test_rand)[mask_unseen.values]

# Need predictions for unseen rows
# If yhat_rand_all aligned to X_test_rand order, subset by mask
yhat_rand_unseen = np.asarray(yhat_rand_all)[mask_unseen.values]
rand_unseen_metrics = calculate_metrics(y_test_unseen, yhat_rand_unseen)

# Group-aware split by site_id (new exp)
# Build full X/y from the random-split parts (same preprocessing/features)
X_full = pd.concat([X_train_rand, X_test_rand], axis=0)
y_full = pd.concat([pd.Series(y_train_rand, index=X_train_rand.index),
                    pd.Series(y_test_rand,  index=X_test_rand.index)], axis=0)

groups_full = ensure_site_id(X_full, decimals=SITE_DECIMALS)

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X_full, y_full, groups=groups_full))

X_train_g = X_full.iloc[train_idx]
y_train_g = y_full.iloc[train_idx]
X_test_g  = X_full.iloc[test_idx]
y_test_g  = y_full.iloc[test_idx]

group_split_metrics, _ = evaluate_model(
    clone(base_est), X_train_g, y_train_g, X_test_g, y_test_g
)

# Summarize in a small table (paper-ready)
summary_rows = [
    {
        "Evaluation": "Random split (all test rows)",
        "n_test_rows": len(X_test_rand),
        "n_test_sites": site_test.nunique(),
        **rand_all_metrics
    },
    {
        "Evaluation": "Random split (unseen-site rows only)",
        "n_test_rows": int(mask_unseen.sum()),
        "n_test_sites": site_test[mask_unseen].nunique(),
        **rand_unseen_metrics
    },
    {
        "Evaluation": "Group-aware split by site (hold-out sites)",
        "n_test_rows": len(X_test_g),
        "n_test_sites": ensure_site_id(X_test_g, decimals=SITE_DECIMALS).nunique(),
        **group_split_metrics
    },
]

spatial_generalization_table = pd.DataFrame(summary_rows)

# Keep only the core metrics you report in the paper (adjust if your calculate_metrics returns more)
keep_cols = ["Evaluation", "n_test_rows", "n_test_sites", "R2", "MAE", "MAPE", "sMAPE", "RMSE"]
keep_cols = [c for c in keep_cols if c in spatial_generalization_table.columns]
spatial_generalization_table = spatial_generalization_table[keep_cols]

display(spatial_generalization_table)

print("\nNotes:")
print(f"- BEST_MODEL_NAME = {BEST_MODEL_NAME}")
print(f"- site_id built from rounded(lat,lon) with decimals={SITE_DECIMALS} (or province fallback if no coords).")
print("- If any target-derived features exist (e.g., per-location mean yield), recompute them inside each split to avoid leakage.")

## Crop-Specific Model Evaluation

In [ ]:
wide_table_with_crops = []

# Iterate through each model and its splits
for model, splits in grouped_results.items():
    # Iterate through each split (e.g., 'Random Split Crop', 'Temporal Split Crop')
    for split, data in splits.items():
        # Iterate through each crop
        for crop_index, metrics in enumerate(data):
            # Initialize a row dictionary for the current crop
            row = {'Model': model, 'Split': split, 'Crop': crop_index}
            # Add metrics to the row
            row.update(metrics)
            # Append the row to the wide_table
            wide_table_with_crops.append(row)

# Convert the table list into a DataFrame
wide_results_table_with_crops = pd.DataFrame(wide_table_with_crops)

# Pivot the DataFrame to the desired wide format
wide_results_table = wide_results_table_with_crops.pivot(
    index=['Model', 'Crop'],
    columns='Split',
    values=['R2', 'MAE', 'MAPE', 'sMAPE', 'RMSE']
)

# Flatten the MultiIndex columns
wide_results_table.columns = [
    f"{metric}_{split}" for metric, split in wide_results_table.columns
]

# Reset index to make it a clean DataFrame
wide_results_table.reset_index(inplace=True)
wide_results_table

# Ensemble Learning

In [ ]:
# Define helper function to load models
def load_model(filepath):
    import joblib
    import tensorflow as tf
    if filepath.endswith(".pkl"):
        return joblib.load(filepath)
    elif filepath.endswith(".keras"):
        # compile=False is fine for inference; keeps things robust
        return tf.keras.models.load_model(
            filepath,
            custom_objects={"PinballLoss": PinballLoss, "qloss": qloss},
            compile=False
        )
    else:
        raise ValueError(f"Unsupported file format for {filepath}")

EPS = 1e-9

def safe_inverse_weights(scores):
    """Inverse-weighting with protection against 0/inf and renormalization."""
    arr = np.asarray(scores, dtype=np.float64)
    arr = np.where(arr <= 0, EPS, arr)              # avoid 1/0
    inv = 1.0 / arr
    inv = np.nan_to_num(inv, nan=0.0, posinf=0.0, neginf=0.0)
    s = inv.sum()
    return (inv / s) if s > 0 else np.ones_like(inv) / len(inv)

def predict_1d(model, X):
    """Predict and return a clean 1-D float64 vector; raise if non-finite."""
    y = model.predict(X)
    y = np.asarray(y, dtype=np.float64).ravel()
    if not np.isfinite(y).all():
        raise ValueError("non-finite predictions")
    return y

def to_1d_pred(yhat, n_expected):
    yhat = np.asarray(yhat)
    if yhat.ndim == 2 and yhat.shape[1] == 1:        # (n,1) -> (n,)
        yhat = yhat.ravel()
    elif yhat.ndim == 2 and yhat.shape[0] == yhat.shape[1] == n_expected:
        yhat = np.diag(yhat)                         # (n,n) -> diag(n,)
    elif yhat.ndim > 1:
        yhat = yhat.reshape(-1)
    return yhat[:n_expected].astype(float)

ensemble_results_crops = []

## Unified Ensemble Learning

In [ ]:
ensemble_sizes = list(range(2, 6))
ensemble_results = []

for n in ensemble_sizes:
    # pick top-N by TRAIN sMAPE (NOT test)
    sorted_models_rand = sorted(
        {k: v for k, v in results.items() if "Ensemble" not in k}.items(),
        key=lambda x: x[1]["Random Train"]["sMAPE"]
    )[:n]
    sorted_models_temp = sorted(
        {k: v for k, v in results.items() if "Ensemble" not in k}.items(),
        key=lambda x: x[1]["Temporal Train"]["sMAPE"]
    )[:n]

    top_model_names_rand = [m[0] for m in sorted_models_rand]
    top_model_names_temp = [m[0] for m in sorted_models_temp]

    # weights from TRAIN sMAPE (NOT test)
    weights_rand = safe_inverse_weights(
        [results[m]["Random Train"]["sMAPE"] for m in top_model_names_rand]
    )
    weights_temp = safe_inverse_weights(
        [results[m]["Temporal Train"]["sMAPE"] for m in top_model_names_temp]
    )

    # --- RANDOM SPLIT ENSEMBLE ---
    preds_rand, used_w_rand, used_names_rand = [], [], []
    for w, model_name in zip(weights_rand, top_model_names_rand):
        model_path = f"models/{model_name}_random_split"
        if os.path.exists(f"{model_path}.pkl"):
            model = load_model(f"{model_path}.pkl")
        elif os.path.exists(f"{model_path}.keras"):
            model = load_model(f"{model_path}.keras")
        else:
            print(f"⚠️ Not found: {model_path}.pkl/.keras")
            continue
        try:
            y_hat = predict_1d(model, X_test_rand)
            preds_rand.append(y_hat)
            used_w_rand.append(w)
            used_names_rand.append(model_name)
        except Exception as e:
            print(f"⚠️ Skipping {model_name} (random): {e}")

    if preds_rand:
        used_w_rand = np.asarray(used_w_rand, dtype=np.float64)
        used_w_rand /= used_w_rand.sum()  # renormalize to 1
        stack = np.vstack(preds_rand)     # shape (k, n)
        y_pred_ensemble_rand = np.average(stack, axis=0, weights=used_w_rand)
    else:
        y_pred_ensemble_rand = np.full_like(y_test_rand, fill_value=np.nan, dtype=np.float64)

    # --- TEMPORAL SPLIT ENSEMBLE ---
    preds_temp, used_w_temp, used_names_temp = [], [], []
    for w, model_name in zip(weights_temp, top_model_names_temp):
        model_path = f"models/{model_name}_temporal_split"
        if os.path.exists(f"{model_path}.pkl"):
            model = load_model(f"{model_path}.pkl")
        elif os.path.exists(f"{model_path}.keras"):
            model = load_model(f"{model_path}.keras")
        else:
            print(f"⚠️ Not found: {model_path}.pkl/.keras")
            continue
        try:
            y_hat = predict_1d(model, X_test_temp)
            preds_temp.append(y_hat)
            used_w_temp.append(w)
            used_names_temp.append(model_name)
        except Exception as e:
            print(f"⚠️ Skipping {model_name} (temporal): {e}")

    if preds_temp:
        used_w_temp = np.asarray(used_w_temp, dtype=np.float64)
        used_w_temp /= used_w_temp.sum()
        stack = np.vstack(preds_temp)
        y_pred_ensemble_temp = np.average(stack, axis=0, weights=used_w_temp)
    else:
        y_pred_ensemble_temp = np.full_like(y_test_temp, fill_value=np.nan, dtype=np.float64)

    # final safety: replace any remaining non-finite with the median of y_true
    y_pred_ensemble_rand = np.nan_to_num(y_pred_ensemble_rand, nan=float(np.nanmedian(y_test_rand)))
    y_pred_ensemble_temp = np.nan_to_num(y_pred_ensemble_temp, nan=float(np.nanmedian(y_test_temp)))

    # metrics
    ensemble_rand_metrics = calculate_metrics(y_test_rand, y_pred_ensemble_rand)
    ensemble_temp_metrics = calculate_metrics(y_test_temp, y_pred_ensemble_temp)

    ensemble_results.append({
        'Model': f"Ensemble_{n}",
        'Number of Models': n,
        'Model Names (Random Split)': ', '.join(used_names_rand),
        'Model Names (Temporal Split)': ', '.join(used_names_temp),
        **{f"{m}_Random Split": v for m, v in ensemble_rand_metrics.items()},
        **{f"{m}_Temporal Split": v for m, v in ensemble_temp_metrics.items()}
    })

ensemble_results_df = pd.DataFrame(ensemble_results)

## Diebold-Mariano Test of Ensemble Learning 

In [ ]:
# Perform Diebold-Mariano Test for Top Ensembles for Random and Temporal Splits
dm_test_results = []

BASE_DIR = "/kaggle/working/models" 

# Define DM Test function
def dm_test(y_true, y_pred1, y_pred2, h=1, loss="sMAPE"):
    """
    Perform Diebold-Mariano test.

    Parameters:
    - y_true: Array-like, true values.
    - y_pred1: Array-like, predictions from Model 1.
    - y_pred2: Array-like, predictions from Model 2.
    - h: Int, forecast horizon (default=1).
    - loss: String, loss function ('MSE' or 'MAE').

    Returns:
    - DM statistic and p-value.
    """
    
    e1 = y_true - y_pred1
    e2 = y_true - y_pred2
    d = e1**2 - e2**2 if loss == "sMAPE" else np.abs(e1) - np.abs(e2)
    d_mean = np.mean(d)
    n = len(d)
    gamma = np.zeros(h)
    for lag in range(h):
        gamma[lag] = np.sum((d[:-lag] - d_mean) * (d[lag:] - d_mean)) / (n - lag) if lag > 0 else np.var(d)
    dm_stat = d_mean / np.sqrt((gamma[0] + 2 * np.sum(gamma[1:])) / n)
    p_value = 2 * (1 - norm.cdf(np.abs(dm_stat)))
    return dm_stat, p_value


def variants(model_name: str, split: str):
    """Generate candidate filenames for robustness."""
    exact = f"{model_name}_{split}"
    us    = exact.replace(' ', '_')
    lower = exact.lower()
    lower_us = lower.replace(' ', '_')
    return [exact, us, lower, lower_us]

def find_model_file(model_name: str, split: str):
    """Try .pkl then .keras across several name variants and return first match (full path)."""
    for stem in variants(model_name, split):
        for ext in (".pkl", ".keras"):
            path = os.path.join(BASE_DIR, stem + ext)
            if os.path.exists(path):
                return path
    # fallback: glob case-insensitive on split
    pattern = os.path.join(BASE_DIR, f"*{split}.*")
    for p in glob.glob(pattern):
        # crude match: contains model name tokens ignoring case/underscores
        mnorm = re.sub(r'[^a-z0-9]+','', model_name.lower())
        pnorm = re.sub(r'[^a-z0-9]+','', os.path.basename(p).lower())
        if mnorm in pnorm:
            return p
    return None

def load_any_model(path: str):
    if path.endswith(".pkl"):
        import joblib
        return joblib.load(path), 'sk'
    elif path.endswith(".keras"):
        return keras.models.load_model(path), 'keras'
    else:
        return None, None

def predict_any(model, kind, X):
    X_ = X.values if hasattr(X, "values") else X
    yhat = model.predict(X_, verbose=0) if kind == 'keras' else model.predict(X_)
    yhat = np.asarray(yhat)
    if yhat.ndim == 2 and yhat.shape[1] == 1:
        yhat = yhat.ravel()
    elif yhat.ndim == 2 and yhat.shape[0] == yhat.shape[1]:
        yhat = np.diag(yhat)  # guard against accidental (n,n)
    elif yhat.ndim > 1:
        yhat = yhat.reshape(-1)
    return yhat

def get_ensemble_predictions(models, X_test, split_type):
    preds = {}
    for name in models:
        path = find_model_file(name, split_type)
        print(path or f"⚠️ Not found for {name} / {split_type}")
        if not path: 
            continue
        model, kind = load_any_model(path)
        preds[name] = predict_any(model, kind, X_test)
    return preds

# Extract top #1 ensemble for Random and Temporal splits
top_random_ensemble = ensemble_results_df.iloc[0]  # First row corresponds to top ensemble for Random Split
top_temporal_ensemble = ensemble_results_df.iloc[0]  # First row corresponds to top ensemble for Temporal Split

# Parse model names for each ensemble
top_random_models = top_random_ensemble['Model Names (Random Split)'].split(', ')
top_temporal_models = top_temporal_ensemble['Model Names (Temporal Split)'].split(', ')

# Predictions dictionary for ensembles
model_predictions = {
    'Random Split': {},
    'Temporal Split': {}
}

# top_random_models, top_temporal_models as before
pred_random   = get_ensemble_predictions(top_random_models,   X_test_rand, 'random_split')
pred_temporal = get_ensemble_predictions(top_temporal_models, X_test_temp,  'temporal_split')

# Load predictions for Random and Temporal splits
model_predictions['Random Split'] = pred_random
model_predictions['Temporal Split'] = pred_temporal

# DM Test for all model pairs within each split
for split_type, y_test, predictions in [("Random Split", y_test_rand, model_predictions['Random Split']), 
                                        ("Temporal Split", y_test_temp, model_predictions['Temporal Split'])]:
    model_names = list(predictions.keys())
    for i, model_name_1 in enumerate(model_names):
        for j, model_name_2 in enumerate(model_names):
            if i >= j:  # Avoid duplicate comparisons and self-comparisons
                continue

            # Get predictions for the two models
            y_pred1 = predictions[model_name_1]
            y_pred2 = predictions[model_name_2]

            # Calculate DM test statistics
            dm_stat, p_value = dm_test(y_test, y_pred1, y_pred2, loss="sMAPE")

            # Store results
            dm_test_results.append({
                'Model 1': model_name_1,
                'Model 2': model_name_2,
                'Split': split_type,
                'DM Statistic': dm_stat,
                'P-Value': p_value
            })

# Convert results to DataFrame
dm_test_results_df = pd.DataFrame(dm_test_results)

# Display DM Test Results Table
dm_test_results_df.sort_values(by=['Split', 'P-Value'], inplace=True)
dm_test_results_df

## Crop-Specific Ensemble Model Evaluation

In [ ]:
for n in ensemble_sizes:
    for crop in wide_results_table_with_crops["Crop"].unique():
        crop_results = wide_results_table_with_crops[wide_results_table_with_crops["Crop"] == crop]

        # Select top N models using TRAIN crop sMAPE (NOT test)
        sorted_models_rand = crop_results[crop_results["Split"] == "Random Train Crop"].nsmallest(n, "sMAPE")
        sorted_models_temp = crop_results[crop_results["Split"] == "Temporal Train Crop"].nsmallest(n, "sMAPE")

        top_model_names_rand = sorted_models_rand["Model"].tolist()
        top_model_names_temp = sorted_models_temp["Model"].tolist()

        # safer weights (inverse sMAPE with guard)
        weights_rand = safe_inverse_weights(sorted_models_rand["sMAPE"].tolist())
        weights_temp = safe_inverse_weights(sorted_models_temp["sMAPE"].tolist())

        X_test_rand_crop = X_test_rand[X_test_rand["crop"] == crop]
        X_test_temp_crop = X_test_temp[X_test_temp["crop"] == crop]
        y_test_rand_crop = y_test_rand[X_test_rand["crop"] == crop]
        y_test_temp_crop = y_test_temp[X_test_temp["crop"] == crop]

        y_pred_ensemble_rand = np.zeros_like(y_test_rand_crop, dtype=float)
        y_pred_ensemble_temp = np.zeros_like(y_test_temp_crop, dtype=float)

        # Aggregate predictions for Random Split (evaluate on test, OK)
        for i, model_name in enumerate(top_model_names_rand):
            model_path = f"models/{model_name}_random_split"
            if os.path.exists(f"{model_path}.pkl"):
                model = load_model(f"{model_path}.pkl")
            elif os.path.exists(f"{model_path}.keras"):
                model = load_model(f"{model_path}.keras")
            else:
                continue

            y_pred_rand = model.predict(X_test_rand_crop)
            y_pred_rand = to_1d_pred(y_pred_rand, len(y_test_rand_crop))
            y_pred_ensemble_rand += weights_rand[i] * y_pred_rand

        # Aggregate predictions for Temporal Split (evaluate on test, OK)
        for i, model_name in enumerate(top_model_names_temp):
            model_path = f"models/{model_name}_temporal_split"
            if os.path.exists(f"{model_path}.pkl"):
                model = load_model(f"{model_path}.pkl")
            elif os.path.exists(f"{model_path}.keras"):
                model = load_model(f"{model_path}.keras")
            else:
                continue

            y_pred_temp = model.predict(X_test_temp_crop)
            y_pred_temp = to_1d_pred(y_pred_temp, len(y_test_temp_crop))
            y_pred_ensemble_temp += weights_temp[i] * y_pred_temp

        # Calculate metrics for Random and Temporal Splits (on test, OK)
        ensemble_rand_metrics = calculate_metrics(y_test_rand_crop, y_pred_ensemble_rand)
        ensemble_temp_metrics = calculate_metrics(y_test_temp_crop, y_pred_ensemble_temp)

        ensemble_results_crops.append({
            "Model": f"Ensemble_{n}",
            "Crop": crop,
            "Number of Models": n,
            "Model Names (Random Split)": ", ".join(top_model_names_rand),
            "Model Names (Temporal Split)": ", ".join(top_model_names_temp),
            **{f"{metric}_Random Split Crop": value for metric, value in ensemble_rand_metrics.items()},
            **{f"{metric}_Temporal Split Crop": value for metric, value in ensemble_temp_metrics.items()},
        })

ensemble_results_crops_df = pd.DataFrame(ensemble_results_crops)

# Feature Importance and Interpretability Analysis
## Feature Analysis

In [ ]:
# Set flag to include/exclude temporal models in analysis
include_temporal = True  # Set to False to exclude temporal models

# Filter top models (exclude ensemble models and stacking models) for Random Split
top_models_random = results_pivot[
    ~results_pivot["Model"].str.contains("Ensemble") & ~results_pivot["Model"].str.contains("Stacking")
]
top_models_random = top_models_random.nsmallest(1, "sMAPE_Random Split")  # Top 3 for Random Split

# Filter top models for Temporal Split if enabled
if include_temporal:
    top_models_temporal = results_pivot[
        ~results_pivot["Model"].str.contains("Ensemble") & ~results_pivot["Model"].str.contains("Stacking")
    ]
    top_models_temporal = top_models_temporal.nsmallest(1, "sMAPE_Temporal Split")  # Top 3 for Temporal Split

# Combine the models while maintaining uniqueness
unique_top_models = set(top_models_random["Model"])
if include_temporal:
    unique_top_models.update(top_models_temporal["Model"])

# Placeholder for storing analysis results
feature_analysis_results = {}

# Loop through unique models for analysis
for model_name in unique_top_models:
    print(f"\nEvaluating {model_name}")
    model_path_random = f"models/{model_name}_random_split.pkl"
    model_path_temporal = f"models/{model_name}_temporal_split.pkl"

    # Initialize model variables
    model_random = None
    model_temporal = None

    # Load models safely
    try:
        if os.path.exists(model_path_random):
            model_random = joblib.load(model_path_random)
        if include_temporal and os.path.exists(model_path_temporal):
            model_temporal = joblib.load(model_path_temporal)
    except Exception as e:
        print(f"Error loading model {model_name}: {e}")
        continue

    try:
        # Feature Importance
        feature_importance_random = (
            model_random.feature_importances_ if model_random and hasattr(model_random, "feature_importances_") else None
        )
        feature_importance_temporal = (
            model_temporal.feature_importances_ if model_temporal and hasattr(model_temporal, "feature_importances_") else None
        ) if include_temporal else None

        # Permutation Importance
        perm_importance_random = permutation_importance(
            model_random, X_test_rand, y_test_rand, scoring='neg_mean_absolute_error'
        ) if model_random else None
        perm_importance_temporal = (
            permutation_importance(model_temporal, X_test_temp, y_test_temp, scoring='neg_mean_absolute_error')
            if include_temporal and model_temporal
            else None
        )

        # SHAP Analysis
        explainer_random = shap.TreeExplainer(model_random) if model_random else None
        shap_values_random = explainer_random.shap_values(X_test_rand) if explainer_random else None

        explainer_temporal = shap.TreeExplainer(model_temporal) if include_temporal and model_temporal else None
        shap_values_temporal = explainer_temporal.shap_values(X_test_temp) if explainer_temporal else None

        # Store results
        feature_analysis_results[model_name] = {
            "feature_importance_random": feature_importance_random,
            "feature_importance_temporal": feature_importance_temporal,
            "perm_importance_random": perm_importance_random.importances_mean if perm_importance_random else None,
            "perm_importance_temporal": perm_importance_temporal.importances_mean if perm_importance_temporal else None,
            "shap_values_random": shap_values_random,
            "shap_values_temporal": shap_values_temporal,
        }
    except Exception as e:
        print(f"Error analyzing model {model_name}: {e}")
        continue

In [ ]:
# Create a directory to save plots if it doesn't already exist
feature_analysis_dir = "feature_analysis"
os.makedirs(feature_analysis_dir, exist_ok=True)

# Define the number of top features to display
TOP_FEATURES = 25

# Loop through top models for visualization
for model_name, results in feature_analysis_results.items():
    # RANDOM SPLIT ANALYSIS
    if results.get("feature_importance_random") is not None:
        # Feature Importance Plot (Random Split)
        sorted_idx_random = np.argsort(-results["feature_importance_random"])[:TOP_FEATURES]
        feature_names_random = X_train_rand.columns[sorted_idx_random]
        fig, ax = plt.subplots(figsize=(12, 8))
        sns.barplot(
            x=results["feature_importance_random"][sorted_idx_random],
            y=feature_names_random,
            ax=ax
        )
        ax.set_title(f"Feature Importance - {model_name} - Random Split")
        ax.set_xlabel("Importance Score")
        ax.set_ylabel("Features")
        plt.tight_layout()
        plot_path = os.path.join(feature_analysis_dir, f"{model_name}_feature_importance_random.png")
        plt.savefig(plot_path)
        plt.show()
        plt.close()

    if results.get("perm_importance_random") is not None:
        # Permutation Importance Plot (Random Split)
        perm_sorted_idx_random = np.argsort(-results["perm_importance_random"])[:TOP_FEATURES]
        feature_names_perm_random = X_train_rand.columns[perm_sorted_idx_random]
        fig, ax = plt.subplots(figsize=(12, 8))
        sns.barplot(
            x=results["perm_importance_random"][perm_sorted_idx_random],
            y=feature_names_perm_random,
            ax=ax
        )
        ax.set_title(f"Permutation Importance - {model_name} - Random Split")
        ax.set_xlabel("Importance Score")
        ax.set_ylabel("Features")
        plt.tight_layout()
        plot_path = os.path.join(feature_analysis_dir, f"{model_name}_permutation_importance_random.png")
        plt.savefig(plot_path)
        plt.show()
        plt.close()

    if results.get("shap_values_random") is not None:
        # SHAP Summary Plot (Random Split)
        fig, ax = plt.subplots(figsize=(12, 8))
        shap.summary_plot(
            results["shap_values_random"], X_test_rand, max_display=TOP_FEATURES, show=False
        )
        plt.title(f"SHAP Summary - {model_name} - Random Split")
        plot_path = os.path.join(feature_analysis_dir, f"{model_name}_shap_random.png")
        plt.savefig(plot_path)
        plt.show()
        plt.close()

    # TEMPORAL SPLIT ANALYSIS (if temporal models are enabled)
    if include_temporal:
        if results.get("feature_importance_temporal") is not None:
            # Feature Importance Plot (Temporal Split)
            sorted_idx_temporal = np.argsort(-results["feature_importance_temporal"])[:TOP_FEATURES]
            feature_names_temporal = X_train_temp.columns[sorted_idx_temporal]
            fig, ax = plt.subplots(figsize=(12, 8))
            sns.barplot(
                x=results["feature_importance_temporal"][sorted_idx_temporal],
                y=feature_names_temporal,
                ax=ax
            )
            ax.set_title(f"Feature Importance - {model_name} - Temporal Split")
            ax.set_xlabel("Importance Score")
            ax.set_ylabel("Features")
            plt.tight_layout()
            plot_path = os.path.join(feature_analysis_dir, f"{model_name}_feature_importance_temporal.png")
            plt.savefig(plot_path)
            plt.show()
            plt.close()

        if results.get("perm_importance_temporal") is not None:
            # Permutation Importance Plot (Temporal Split)
            perm_sorted_idx_temporal = np.argsort(-results["perm_importance_temporal"])[:TOP_FEATURES]
            feature_names_perm_temporal = X_train_temp.columns[perm_sorted_idx_temporal]
            fig, ax = plt.subplots(figsize=(12, 8))
            sns.barplot(
                x=results["perm_importance_temporal"][perm_sorted_idx_temporal],
                y=feature_names_perm_temporal,
                ax=ax
            )
            ax.set_title(f"Permutation Importance - {model_name} - Temporal Split")
            ax.set_xlabel("Importance Score")
            ax.set_ylabel("Features")
            plt.tight_layout()
            plot_path = os.path.join(feature_analysis_dir, f"{model_name}_permutation_importance_temporal.png")
            plt.savefig(plot_path)
            plt.show()
            plt.close()

        if results.get("shap_values_temporal") is not None:
            # SHAP Summary Plot (Temporal Split)
            fig, ax = plt.subplots(figsize=(12, 8))
            shap.summary_plot(
                results["shap_values_temporal"], X_test_temp, max_display=TOP_FEATURES, show=False
            )
            plt.title(f"SHAP Summary - {model_name} - Temporal Split")
            plot_path = os.path.join(feature_analysis_dir, f"{model_name}_shap_temporal.png")
            plt.savefig(plot_path)
            plt.show()
            plt.close()

In [ ]:
# Prepare a list to hold feature importance data for all models
feature_analysis_data = []

# Loop through each model and extract feature importance results
for model_name, results in feature_analysis_results.items():
    # Random Split Feature Importance
    for i, feature_name in enumerate(X_train_rand.columns):
        feature_analysis_data.append({
            "Model": model_name,
            "Split": "Random",
            "Feature": feature_name,
            "Importance": results["feature_importance_random"][i]
        })
    
    # Temporal Split Feature Importance
    for i, feature_name in enumerate(X_train_temp.columns):
        feature_analysis_data.append({
            "Model": model_name,
            "Split": "Temporal",
            "Feature": feature_name,
            "Importance": results["feature_importance_temporal"][i]
        })

# Convert the list of dictionaries to a DataFrame
feature_analysis_df = pd.DataFrame(feature_analysis_data)
feature_analysis_df.sort_values(by="Importance", ascending=False)

## Random-Temporal Importance Comparison

In [ ]:
# Calculate average importance per feature for each split
random_split_importance = feature_analysis_df[feature_analysis_df['Split'] == 'Random'].groupby('Feature')['Importance'].mean()
temporal_split_importance = feature_analysis_df[feature_analysis_df['Split'] == 'Temporal'].groupby('Feature')['Importance'].mean()

# Combine into a single DataFrame for comparison
importance_comparison = pd.DataFrame({
    'Feature': random_split_importance.index,
    'Random Importance': random_split_importance.values,
    'Temporal Importance': temporal_split_importance.reindex(random_split_importance.index).values
})

# Calculate the difference and rank features by the difference
importance_comparison['Difference'] = importance_comparison['Random Importance'] - importance_comparison['Temporal Importance']
importance_comparison['Absolute Difference'] = importance_comparison['Difference'].abs()

# Sort by absolute difference for analysis
importance_comparison = importance_comparison.sort_values(by='Absolute Difference', ascending=False)
importance_comparison.sort_values(by="Absolute Difference", ascending=False)